In [ ]:
!pip install transformers datasets evaluate accelerate

In [ ]:
from datasets import load_dataset
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
!pip install evaluate
import evaluate

In [ ]:
print(transformers.__version__)
print(transformers.__file__)

In [ ]:
dataset = load_dataset("nkazi/SciEntsBank")

# use UA split (unseen answers)
train_ds = dataset["train"]
test_ds = dataset["test_ua"]

In [ ]:
# model_name = "distilbert-base-uncased"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name) # tokenization 把句子切成单元词

In [ ]:
# Preprocess function: combine question + student_answer
def preprocess(data):
    # 把文本转换成了 input_ids, attention_mask, token_type_ids
    return tokenizer(
        data["question"],
        data["student_answer"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

encoded_train = train_ds.map(preprocess, batched=True) # 文字转成数字的token tokenization + encoding（把token词转成数字IDs）
encoded_test = test_ds.map(preprocess, batched=True)

In [ ]:
# define model
num_labels = 5   # correct, wrong, partially_correct, irrelevant, contradictory
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

In [ ]:
# metric
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels),
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro"),
        "f1_weighted": f1.compute(predictions=preds, references=labels, average="weighted"),
    }

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
save_path = "/content/drive/MyDrive/Misconception_Analysis/sci_bert_model"



In [ ]:
args = TrainingArguments(
    output_dir=save_path,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    save_steps=500,                  # 每500步保存一次
    save_total_limit=2,              # 只保留最近2个checkpoint
    eval_steps=500,                  # 每500步做一次eval
    logging_steps=100,               # 每100步打印一次日志
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)


In [ ]:
# trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
#Train & Evaluate
trainer.train()
results = trainer.evaluate()
print(results)

In [ ]:

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

Model Evaluation and Analysis

The bert-base-uncased model was evaluated on the SciEntsBank dataset using accuracy and F1 scores. The results after three training epochs are summarized as follows:

Evaluation Loss: 1.09

Accuracy: 53.7%

Macro F1: 0.375

Weighted F1: 0.529

These results suggest that the model performs substantially better than random guessing (baseline ≈ 20% with five classes), indicating that it has learned meaningful patterns in student responses. However, there are clear differences between the metrics:

Accuracy vs. Macro F1: While overall accuracy is moderate (53.7%), the lower macro F1 score (0.375) reveals that the model struggles with minority classes, such as irrelevant or contradictory responses. This reflects the class imbalance in the dataset.

Weighted F1: The weighted F1 score (0.529) is closer to accuracy, suggesting that performance is stronger on the more frequent classes (correct, wrong) but weaker on the less represented ones.

From an educational perspective, this indicates that the model is relatively reliable at identifying clearly correct or incorrect responses but has more difficulty distinguishing categories such as partially correct and contradictory. Improving performance on these minority categories is critical for building systems that can effectively diagnose student misconceptions.

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

# model = AutoModelForSequenceClassification.from_pretrained(
#     "/content/drive/MyDrive/Misconception_Analysis/sci_bert_model"
# )
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=encoded_train,
#     eval_dataset=encoded_test,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# trainer.train(resume_from_checkpoint=True)
